In [1]:
import pandas as pd
import numpy as np
import random
import string 
import tensorflow as tf
import cv2

In [2]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


# Generate CAPTCHA images

### Install Python's captcha library

In [3]:
!pip install captcha

In [4]:
from captcha.image import ImageCaptcha

### CAPTCHA images 

 the function generates images  include 5 characters :
 - letters and numbers
 -  symbols and punctuations (optional) 

input:
- ratio  
    - letters : numbers : symbols
- split percentage 

output:
- Dictionary { keys -> train + validate + test , values -> list [ image path' , tag ] }


In [5]:
def CAPTCHAsgenerating (datasize: int , imgsize : tuple , ratio :tuple ,tr_s :float ):
  v_s = float((1-tr_s)/2)
  data_dict = {'tr':[],'val':[],'ts':[]}
  captcha =[]
  image = ImageCaptcha(width = imgsize[0] , height = imgsize[1] )
  punctuation =string.punctuation
  list(punctuation).remove("/")
  for i in range(datasize):
    letters = [random.choice(string.ascii_letters) for i in range(ratio[0]) ]
    numbers = [random.choice(string.digits       ) for i in range(ratio[1]) ]
    symbols = [random.choice(punctuation         ) for i in range(ratio[2]) if not '/']
    captcha_text = [str(elem) for elem in letters+numbers+symbols ] 
    random.shuffle( captcha_text) 
    captcha_text = ''.join(captcha_text)
    path = '/content/images/'+captcha_text+'.png'
    captcha.append ([path,captcha_text])
    data = image.generate(captcha_text) 
    image.write(captcha_text, path )
    tr, val , ts = np.split(captcha, [int(len(captcha)*tr_s), int(len(captcha)*(1-v_s)) ])
    data_dict['tr'].extend(tr)
    data_dict['val'].extend(val)
    data_dict['ts'].extend(ts)
    
  return data_dict

In [6]:
data_dict= CAPTCHAsgenerating(datasize= 20  , imgsize =(280 , 90) , ratio = (2,2,1) ,tr_s =0.8 )

In [ ]:
print (data_dict['val'])